## EDA and Feature Engineering — Flight Price Prediction

### **Features**

The cleaned dataset contains the following features:

1. **Airline**  
   Represents the airline company. This is a categorical feature and contains 6 unique airline names.

2. **Flight**  
   Contains the flight code information. It is a categorical feature.

3. **Source City**  
   Indicates the city from which the flight departs. This is a categorical feature with 6 distinct cities.

4. **Departure Time**  
   A derived categorical feature created by grouping departure times into time intervals (bins). Contains 6 unique time labels.

5. **Stops**  
   A categorical feature with 3 distinct values showing how many stops the flight has between source and destination (e.g., non-stop, 1 stop, 2+ stops).

6. **Arrival Time**  
   A derived categorical feature created by grouping arrival times into time intervals. Contains 6 unique labels.

7. **Destination City**  
   Represents the city where the flight lands. It is a categorical feature with 6 unique destination cities.

8. **Class**  
   Indicates the seat class. It is a categorical feature with 2 categories: Business and Economy.

9. **Duration**  
   A continuous numeric feature showing the total travel time between cities in hours.

10. **Days Left**  
   A derived numeric feature calculated as the difference between the booking date and the travel date (how many days before the flight was booked).

11. **Price**
    A target variable stores information of ticket price.

In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [5]:
df=pd.read_excel("flight_data.xlsx")

In [6]:
df.head()

,Airline,Date_of_Journey,Source,Destination,Route,Dep_Time,Arrival_Time,Duration,Total_Stops,Additional_Info,Price
0,IndiGo,24/03/2019,Banglore,New Delhi,BLR → DEL,22:20,01:10 22 Mar,2h 50m,non-stop,No info,3897
1,Air India,1/05/2019,Kolkata,Banglore,CCU → IXR → BBI → BLR,05:50,13:15,7h 25m,2 stops,No info,7662
2,Jet Airways,9/06/2019,Delhi,Cochin,DEL → LKO → BOM → COK,09:25,04:25 10 Jun,19h,2 stops,No info,13882
3,IndiGo,12/05/2019,Kolkata,Banglore,CCU → NAG → BLR,18:05,23:30,5h 25m,1 stop,No info,6218
4,IndiGo,01/03/2019,Banglore,New Delhi,BLR → NAG → DEL,16:50,21:35,4h 45m,1 stop,No info,13302


In [7]:
## EDA 
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10683 entries, 0 to 10682
Data columns (total 11 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   Airline          10683 non-null  object
 1   Date_of_Journey  10683 non-null  object
 2   Source           10683 non-null  object
 3   Destination      10683 non-null  object
 4   Route            10682 non-null  object
 5   Dep_Time         10683 non-null  object
 6   Arrival_Time     10683 non-null  object
 7   Duration         10683 non-null  object
 8   Total_Stops      10682 non-null  object
 9   Additional_Info  10683 non-null  object
 10  Price            10683 non-null  int64 
dtypes: int64(1), object(10)
memory usage: 918.2+ KB


In [8]:
df.describe()

,Price
count,10683.000000
mean,9087.064121
std,4611.359167
min,1759.000000
25%,5277.000000
50%,8372.000000
75%,12373.000000
max,79512.000000


also there is only one numeric feature price so for machine understanding we have to create numeric data of other columns right

#### in given dataset date of journy is in string so we can create in time series right 
so for that we perform below operation

In [9]:
df['Date']=df['Date_of_Journey'].str.split('/').str[0]
df['Month']=df['Date_of_Journey'].str.split('/').str[1]
df['Year']=df['Date_of_Journey'].str.split('/').str[2]


In [10]:
df.head(2)

,Airline,Date_of_Journey,Source,Destination,Route,Dep_Time,Arrival_Time,Duration,Total_Stops,Additional_Info,Price,Date,Month,Year
0,IndiGo,24/03/2019,Banglore,New Delhi,BLR → DEL,22:20,01:10 22 Mar,2h 50m,non-stop,No info,3897,24,03,2019
1,Air India,1/05/2019,Kolkata,Banglore,CCU → IXR → BBI → BLR,05:50,13:15,7h 25m,2 stops,No info,7662,1,05,2019


In [11]:
## but here date month and yeaar is in string so for numeric
df['Date']=df['Date'].astype(int)
df['Month']=df["Month"].astype(int)
df['Year']=df["Year"].astype(int)

In [12]:
df.head()

,Airline,Date_of_Journey,Source,Destination,Route,Dep_Time,Arrival_Time,Duration,Total_Stops,Additional_Info,Price,Date,Month,Year
0,IndiGo,24/03/2019,Banglore,New Delhi,BLR → DEL,22:20,01:10 22 Mar,2h 50m,non-stop,No info,3897,24,3,2019
1,Air India,1/05/2019,Kolkata,Banglore,CCU → IXR → BBI → BLR,05:50,13:15,7h 25m,2 stops,No info,7662,1,5,2019
2,Jet Airways,9/06/2019,Delhi,Cochin,DEL → LKO → BOM → COK,09:25,04:25 10 Jun,19h,2 stops,No info,13882,9,6,2019
3,IndiGo,12/05/2019,Kolkata,Banglore,CCU → NAG → BLR,18:05,23:30,5h 25m,1 stop,No info,6218,12,5,2019
4,IndiGo,01/03/2019,Banglore,New Delhi,BLR → NAG → DEL,16:50,21:35,4h 45m,1 stop,No info,13302,1,3,2019


#### so now we have numeric data of date of journey so we can remove or drop date of journey column

In [13]:
df.drop(columns='Date_of_Journey',axis=1,inplace=True)

In [14]:
df.head(2)

,Airline,Source,Destination,Route,Dep_Time,Arrival_Time,Duration,Total_Stops,Additional_Info,Price,Date,Month,Year
0,IndiGo,Banglore,New Delhi,BLR → DEL,22:20,01:10 22 Mar,2h 50m,non-stop,No info,3897,24,3,2019
1,Air India,Kolkata,Banglore,CCU → IXR → BBI → BLR,05:50,13:15,7h 25m,2 stops,No info,7662,1,5,2019


#### now we can see in Arrival time somewhere month is show somewhere not so for that we can make it only hour and minutes

In [15]:
df["Arrival_hour"]=(df['Arrival_Time'].str.split(' ').str[0].str.split(":").str[0]).astype(int)
df["Arrival_min"]=(df['Arrival_Time'].str.split(' ').str[0].str.split(":").str[1]).astype(int)


df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10683 entries, 0 to 10682
Data columns (total 15 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   Airline          10683 non-null  object
 1   Source           10683 non-null  object
 2   Destination      10683 non-null  object
 3   Route            10682 non-null  object
 4   Dep_Time         10683 non-null  object
 5   Arrival_Time     10683 non-null  object
 6   Duration         10683 non-null  object
 7   Total_Stops      10682 non-null  object
 8   Additional_Info  10683 non-null  object
 9   Price            10683 non-null  int64 
 10  Date             10683 non-null  int64 
 11  Month            10683 non-null  int64 
 12  Year             10683 non-null  int64 
 13  Arrival_hour     10683 non-null  int64 
 14  Arrival_min      10683 non-null  int64 
dtypes: int64(6), object(9)
memory usage: 1.2+ MB


In [16]:
## now arrival time is nothing needed so drop this column
df.drop("Arrival_Time",axis=1,inplace=True)


In [17]:
df.head(2)

,Airline,Source,Destination,Route,Dep_Time,Duration,Total_Stops,Additional_Info,Price,Date,Month,Year,Arrival_hour,Arrival_min
0,IndiGo,Banglore,New Delhi,BLR → DEL,22:20,2h 50m,non-stop,No info,3897,24,3,2019,1,10
1,Air India,Kolkata,Banglore,CCU → IXR → BBI → BLR,05:50,7h 25m,2 stops,No info,7662,1,5,2019,13,15


In [18]:
## now Dep_Time is also in string so again same above method
df['Dep_hour']=(df['Dep_Time'].str.split(':').str[0]).astype(int)
df["Dep_min"]=(df["Dep_Time"].str.split(':').str[1]).astype(int)
df.drop("Dep_Time",inplace=True,axis=1)

In [19]:
df.head(1)

,Airline,Source,Destination,Route,Duration,Total_Stops,Additional_Info,Price,Date,Month,Year,Arrival_hour,Arrival_min,Dep_hour,Dep_min
0,IndiGo,Banglore,New Delhi,BLR → DEL,2h 50m,non-stop,No info,3897,24,3,2019,1,10,22,20


In [20]:
# here Route is no needed in here so drop this column
df.drop("Route",inplace=True,axis=1)

In [21]:
## now we see Total_Stops is 
df["Total_Stops"].unique()

array(['non-stop', '2 stops', '1 stop', '3 stops', nan, '4 stops'],
      dtype=object)

In [22]:
""" here we can see that some value is missing  since it is category feature so we take mode and 
    replace it by this and then we generate numeric column for it and for machine understanding   """
   

' here we can see that some value is missing  since it is category feature so we take mode and \n    replace it by this and then we generate numeric column for it and for machine understanding   '

**Category variable**   **Numeric convert**

**non stop**                              0

**1 stop**                              1

**2 stop**                              2

**3 stop**                              3

**4 stop**                              4    

**nan stop**                            1

In [23]:
df["Total_Stops"]=df["Total_Stops"].map({'non-stop':0 , '1 stop':1 , '2 stops':2 , '3 stops':3 , '4 stops':4 , np.nan:1})

In [24]:
df.head()

,Airline,Source,Destination,Duration,Total_Stops,Additional_Info,Price,Date,Month,Year,Arrival_hour,Arrival_min,Dep_hour,Dep_min
0,IndiGo,Banglore,New Delhi,2h 50m,0,No info,3897,24,3,2019,1,10,22,20
1,Air India,Kolkata,Banglore,7h 25m,2,No info,7662,1,5,2019,13,15,5,50
2,Jet Airways,Delhi,Cochin,19h,2,No info,13882,9,6,2019,4,25,9,25
3,IndiGo,Kolkata,Banglore,5h 25m,1,No info,6218,12,5,2019,23,30,18,5
4,IndiGo,Banglore,New Delhi,4h 45m,1,No info,13302,1,3,2019,21,35,16,50


In [25]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10683 entries, 0 to 10682
Data columns (total 14 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   Airline          10683 non-null  object
 1   Source           10683 non-null  object
 2   Destination      10683 non-null  object
 3   Duration         10683 non-null  object
 4   Total_Stops      10683 non-null  int64 
 5   Additional_Info  10683 non-null  object
 6   Price            10683 non-null  int64 
 7   Date             10683 non-null  int64 
 8   Month            10683 non-null  int64 
 9   Year             10683 non-null  int64 
 10  Arrival_hour     10683 non-null  int64 
 11  Arrival_min      10683 non-null  int64 
 12  Dep_hour         10683 non-null  int64 
 13  Dep_min          10683 non-null  int64 
dtypes: int64(9), object(5)
memory usage: 1.1+ MB


In [26]:
# destination hour
df['Duration_hour'] = df['Duration'].str.split(
    ' ').str[0].str.split('h').str[0]
df['Duration_hour']

0         2
1         7
2        19
3         5
4         4
         ..
10678     2
10679     2
10680     3
10681     2
10682     8
Name: Duration_hour, Length: 10683, dtype: object

In [27]:
df[df['Duration_hour'] == '5m']

,Airline,Source,Destination,Duration,Total_Stops,Additional_Info,Price,Date,Month,Year,Arrival_hour,Arrival_min,Dep_hour,Dep_min,Duration_hour
6474,Air India,Mumbai,Hyderabad,5m,2,No info,17327,6,3,2019,16,55,16,50,5m


In [28]:
df.drop(6474,axis=0,inplace=True)

In [29]:
df['Duration_hour'] = df['Duration_hour'].astype(int)

In [30]:
df['Duration_hour'] = df['Duration_hour']*60

In [31]:
df['Duration_min'] = df['Duration_hour']

In [32]:
df.drop('Duration_hour', axis=1, inplace=True)

In [33]:
df.drop('Duration', axis=1, inplace=True)

In [34]:
df.head(2)


,Airline,Source,Destination,Total_Stops,Additional_Info,Price,Date,Month,Year,Arrival_hour,Arrival_min,Dep_hour,Dep_min,Duration_min
0,IndiGo,Banglore,New Delhi,0,No info,3897,24,3,2019,1,10,22,20,120
1,Air India,Kolkata,Banglore,2,No info,7662,1,5,2019,13,15,5,50,420


In [35]:
df["Total_Stops"].isnull().sum()
df.head(1)

,Airline,Source,Destination,Total_Stops,Additional_Info,Price,Date,Month,Year,Arrival_hour,Arrival_min,Dep_hour,Dep_min,Duration_min
0,IndiGo,Banglore,New Delhi,0,No info,3897,24,3,2019,1,10,22,20,120


In [36]:
df["Airline"].unique()

array(['IndiGo', 'Air India', 'Jet Airways', 'SpiceJet',
       'Multiple carriers', 'GoAir', 'Vistara', 'Air Asia',
       'Vistara Premium economy', 'Jet Airways Business',
       'Multiple carriers Premium economy', 'Trujet'], dtype=object)

In [37]:
df['Source'].unique()

array(['Banglore', 'Kolkata', 'Delhi', 'Chennai', 'Mumbai'], dtype=object)

In [38]:
df['Destination'].unique()

array(['New Delhi', 'Banglore', 'Cochin', 'Kolkata', 'Delhi', 'Hyderabad'],
      dtype=object)

#### here we can see in above 3 cell each feature has less unique value so we can perform on that OneHotEncoding(OHE) for convert it into numeric for train dataset

In [39]:
from sklearn.preprocessing import OneHotEncoder
encoder=OneHotEncoder()

In [40]:
encoded=encoder.fit_transform(df[['Airline'	,'Source',	'Destination']])

In [ ]:
df_encoded=pd.DataFrame(encoded.toarray(),columns=encoder.get_feature_names_out()) # type: ignore

In [42]:
df.drop(columns={'Airline',	'Source'	,'Destination'},inplace=True)

In [43]:
df['Additional_Info'].unique()

array(['No info', 'In-flight meal not included',
       'No check-in baggage included', '1 Short layover', 'No Info',
       '1 Long layover', 'Change airports', 'Business class',
       'Red-eye flight', '2 Long layover'], dtype=object)

In [44]:
from sklearn.preprocessing import LabelEncoder
labelencoder=LabelEncoder()

In [45]:
df['Additional_Info']=labelencoder.fit_transform(df['Additional_Info'])

In [46]:
df.head()

,Total_Stops,Additional_Info,Price,Date,Month,Year,Arrival_hour,Arrival_min,Dep_hour,Dep_min,Duration_min
0,0,8,3897,24,3,2019,1,10,22,20,120
1,2,8,7662,1,5,2019,13,15,5,50,420
2,2,8,13882,9,6,2019,4,25,9,25,1140
3,1,8,6218,12,5,2019,23,30,18,5,300
4,1,8,13302,1,3,2019,21,35,16,50,240


In [47]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 10682 entries, 0 to 10682
Data columns (total 11 columns):
 #   Column           Non-Null Count  Dtype
---  ------           --------------  -----
 0   Total_Stops      10682 non-null  int64
 1   Additional_Info  10682 non-null  int64
 2   Price            10682 non-null  int64
 3   Date             10682 non-null  int64
 4   Month            10682 non-null  int64
 5   Year             10682 non-null  int64
 6   Arrival_hour     10682 non-null  int64
 7   Arrival_min      10682 non-null  int64
 8   Dep_hour         10682 non-null  int64
 9   Dep_min          10682 non-null  int64
 10  Duration_min     10682 non-null  int64
dtypes: int64(11)
memory usage: 1001.4 KB


In [48]:
final_df=pd.concat([df,df_encoded],axis=1).reset_index(drop=True)

In [49]:
final_df

,Total_Stops,Additional_Info,Price,Date,Month,Year,Arrival_hour,Arrival_min,Dep_hour,Dep_min,...,Source_Chennai,Source_Delhi,Source_Kolkata,Source_Mumbai,Destination_Banglore,Destination_Cochin,Destination_Delhi,Destination_Hyderabad,Destination_Kolkata,Destination_New Delhi
0,0.0,8.0,3897.0,24.0,3.0,2019.0,1.0,10.0,22.0,20.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
1,2.0,8.0,7662.0,1.0,5.0,2019.0,13.0,15.0,5.0,50.0,...,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
2,2.0,8.0,13882.0,9.0,6.0,2019.0,4.0,25.0,9.0,25.0,...,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
3,1.0,8.0,6218.0,12.0,5.0,2019.0,23.0,30.0,18.0,5.0,...,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
4,1.0,8.0,13302.0,1.0,3.0,2019.0,21.0,35.0,16.0,50.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10678,0.0,8.0,4145.0,27.0,4.0,2019.0,23.0,20.0,20.0,45.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
10679,0.0,8.0,7229.0,27.0,4.0,2019.0,11.0,20.0,8.0,20.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
10680,0.0,8.0,12648.0,1.0,3.0,2019.0,14.0,10.0,11.0,30.0,...,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
10681,2.0,8.0,11753.0,9.0,5.0,2019.0,19.0,15.0,10.0,55.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
